In [2]:
from nltk.tokenize import RegexpTokenizer, word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv('spam.csv', encoding='ISO-8859-1')
le = LabelEncoder()

In [4]:
data = df.to_numpy()

In [5]:
data

array([['ham',
        'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
        nan, nan, nan],
       ['ham', 'Ok lar... Joking wif u oni...', nan, nan, nan],
       ['spam',
        "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
        nan, nan, nan],
       ...,
       ['ham',
        'Pity, * was in mood for that. So...any other suggestions?', nan,
        nan, nan],
       ['ham',
        "The guy did some bitching but I acted like i'd be interested in buying something else next week and he gave it to us for free",
        nan, nan, nan],
       ['ham', 'Rofl. Its true to its name', nan, nan, nan]], dtype=object)

In [7]:
X = data[:, 1]
y = data[:, 0]

In [8]:
X.shape, y.shape

((5572,), (5572,))

In [9]:
tokenizer = RegexpTokenizer('\w+')
sw = set(stopwords.words('english'))
ps = PorterStemmer()

In [10]:
def getStem(review):
    review = review.lower()
    tokens = tokenizer.tokenize(review) # breaking into small words
    removed_stopwords = [w for w in tokens if w not in sw]
    stemmed_words = [ps.stem(token) for token in removed_stopwords]
    clean_review = ' '.join(stemmed_words)
    return clean_review

In [11]:
# get a clean document
def getDoc(document):
    d = []
    for doc in document:
        d.append(getStem(doc))
    return d

In [12]:
stemmed_doc = getDoc(X)

In [13]:
stemmed_doc[:10]

['go jurong point crazi avail bugi n great world la e buffet cine got amor wat',
 'ok lar joke wif u oni',
 'free entri 2 wkli comp win fa cup final tkt 21st may 2005 text fa 87121 receiv entri question std txt rate c appli 08452810075over18',
 'u dun say earli hor u c alreadi say',
 'nah think goe usf live around though',
 'freemsg hey darl 3 week word back like fun still tb ok xxx std chg send å 1 50 rcv',
 'even brother like speak treat like aid patent',
 'per request mell mell oru minnaminungint nurungu vettam set callertun caller press 9 copi friend callertun',
 'winner valu network custom select receivea å 900 prize reward claim call 09061701461 claim code kl341 valid 12 hour',
 'mobil 11 month u r entitl updat latest colour mobil camera free call mobil updat co free 08002986030']

In [15]:
cv = CountVectorizer()

In [16]:
# create my vocab
vc = cv.fit_transform(stemmed_doc)

In [17]:
X = vc.todense()

In [18]:
X_train, X_test, y_train, y_test = train_test_split(
...     X, y, test_size=0.33, random_state=42)

# NB from sklearn

In [19]:
from sklearn.naive_bayes import MultinomialNB

In [20]:
model = MultinomialNB()
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.977705274605764

In [37]:
messages = [
    """
        Dear All,
        On the occasion of Raksha Bandhan, there will be no classes on Monday, 03 August 2020.
    """,
    """
       Join us today at 12:00 PM ET / 16:00 UTC for a Red Hat DevNation tech talk on AWS Lambda and serverless Java with Bill Burke.
Have you ever tried Java on AWS Lambda but found that the cold-start latency and memory usage were far too high? 
In this session, we will show how we optimized Java for serverless applications by leveraging GraalVM with Quarkus to 
provide both supersonic startup speed and a subatomic memory footprint.
        
    """,
    """
        Within the past few months, we have onboarded 100+ tech colleges and more than 12000+ students pan India.
        Our Campus ambassadors help us build and lead these communities on their campuses and you can apply for this role too.
    """
]

In [38]:
def prepare(messages):
    d = getDoc(messages)
    # dont do fit_transform!! it will create new vocab.
    return cv.transform(d)

messages = prepare(messages)

In [39]:
y_pred = model.predict(messages)
y_pred

array(['ham', 'spam', 'ham'], dtype='<U4')